In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 81.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 17.5 MB/s eta 0:00:00


In [2]:
!pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9674 sha256=c8e7a9e47afbdf28bc948811b17b076754e4d246fd74ef6e48793284ca711d1f
  Stored in directory: /root/.cache/pip/wheels/bd/a8/c3/3cf2c14a1837a4e04bd98631724e81f33f462d86a1d895fae0
Successfully built wget


In [3]:
!pip install colorama

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:

import numpy as np 
import random 
import time
import torch 
import os 
import wget 
import pandas as pd 
import colorama
import tensorflow as tf 

from tensorflow import keras
from transformers import BertTokenizer 
from colorama import Fore
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader , TensorDataset ,RandomSampler , SequentialSampler
from transformers import BertForSequenceClassification  , AdamW , BertConfig
import warnings 
warnings.simplefilter("ignore")
from transformers import get_linear_schedule_with_warmup






In [5]:
device ='cuda' if torch.cuda.is_available() else 'cpu' 

In [6]:

url = 'https://nyu-mll.github.io/CoLA/cola_public_1.1.zip'

if not os.path.exists('./cola_public_1.1.zip'):
    wget.download(url ,'./cola_public_1.1.zip')

In [7]:
if not os.path.exists('./cola_public/'):
    !unzip cola_public_1.1.zip

Archive:  cola_public_1.1.zip
   creating: cola_public/
  inflating: cola_public/README      
   creating: cola_public/tokenized/
  inflating: cola_public/tokenized/in_domain_dev.tsv  
  inflating: cola_public/tokenized/in_domain_train.tsv  
  inflating: cola_public/tokenized/out_of_domain_dev.tsv  
   creating: cola_public/raw/
  inflating: cola_public/raw/in_domain_dev.tsv  
  inflating: cola_public/raw/in_domain_train.tsv  
  inflating: cola_public/raw/out_of_domain_dev.tsv  


In [8]:

train_data = pd.read_csv('./cola_public/raw/in_domain_train.tsv' , sep ='\t' , header = None , 
                        names = ['sentence_source' , 'label' , 'label_notes' ,'sentence'])

In [9]:
train_data.head()

,sentence_source,label,label_notes,sentence
0,gj04,1,NaN,"Our friends won't buy this analysis, let alone..."
1,gj04,1,NaN,One more pseudo generalization and I'm giving up.
2,gj04,1,NaN,One more pseudo generalization or I'm giving up.
3,gj04,1,NaN,"The more we study verbs, the crazier they get."
4,gj04,1,NaN,Day by day the facts are getting murkier.


In [10]:
sentence = train_data['sentence'].values
label = train_data['label'].values

In [11]:

print('Downloading model.....')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased' , do_lower_case =True)

In [12]:
## testing our model 
print(f'Original sentence:\n {sentence[0]}\n ')
print(f'sentence tokens :\n {tokenizer.encode(sentence[0])}')



Original sentence:
 Our friends won't buy this analysis, let alone the next one we propose.
 
sentence tokens :
 [101, 2256, 2814, 2180, 1005, 1056, 4965, 2023, 4106, 1010, 2292, 2894, 1996, 2279, 2028, 2057, 16599, 1012, 102]


In [13]:
## we will do :
# 1. token sentence 
# 2. add special tokens like -->[SEP , CLS]
# 3. turn it to ids 

get_ids = []

for sent in sentence :
    tokens  = tokenizer.encode(sent  , add_special_tokens = True)
    get_ids.append(tokens)


In [14]:

print(f'Original sentence:--> {sentence[0]}')
print(Fore.GREEN +f'Sentence Tokens:----> {get_ids[0]}')

Original sentence:--> Our friends won't buy this analysis, let alone the next one we propose.
Sentence Tokens:----> [101, 2256, 2814, 2180, 1005, 1056, 4965, 2023, 4106, 1010, 2292, 2894, 1996, 2279, 2028, 2057, 16599, 1012, 102]


In [15]:
## here we will adding padding values 

print(Fore.LIGHTBLUE_EX+  f'Max Length of sentence {Fore.GREEN+str(max([len(sent) for sent in get_ids]))}\n')  ### so we will use 64 as max of lenth to matched to processor 


from keras.utils import pad_sequences
max_len = 64 
inputs_ids = pad_sequences(get_ids , maxlen =max_len , dtype = 'long', value = 0 ,
                           padding = 'post' , truncating='post')

Max Length of sentence 47



In [16]:
for i in range(5):
    print(inputs_ids[i])
    print(f'it is lenght is :{len(inputs_ids[i])}\n')

[  101  2256  2814  2180  1005  1056  4965  2023  4106  1010  2292  2894
  1996  2279  2028  2057 16599  1012   102     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0]
it is lenght is :64

[  101  2028  2062 18404  2236  3989  1998  1045  1005  1049  3228  2039
  1012   102     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0]
it is lenght is :64

[  101  2028  2062 18404  2236  3989  2030  1045  1005  1049  3228  2039
  1012   102     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0

# Attention Mask

In [17]:
# makes it explicit which tokens are actual words versus which are padding 
## all values (0 or 1) so 0----> is padding values ,  1----> is real values

attention_mask = []

for ids in inputs_ids :
    mask = [int(tokens>0) for tokens in ids]
    attention_mask.append(mask)

In [18]:
## spliting data 


X_train , X_val , y_train ,y_val = train_test_split(inputs_ids , label , test_size =0.1  
                                                    , random_state=2023)
train_mask , val_mask = train_test_split(attention_mask , test_size=0.1 , random_state=2023 )

# transform data into tensors

In [19]:
#preparation data for torch :
train_input = torch.tensor(X_train) 
val_input = torch.tensor(X_val)
train_label = torch.tensor(y_train)
val_label = torch.tensor(y_val)

tr_mask = torch.tensor(train_mask)
vl_mask = torch.tensor(val_mask)


In [20]:
batch_size = 32

train_data = TensorDataset(train_input , tr_mask ,train_label)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data , sampler=train_sampler , batch_size=batch_size)



validation_data = TensorDataset(val_input ,vl_mask ,val_label)
validation_sampler = RandomSampler(validation_data)
val_dataloader = DataLoader(validation_data ,sampler =validation_sampler ,batch_size=batch_size)

In [21]:

# Calling model from Hugging Face 
model  = BertForSequenceClassification.from_pretrained('bert-base-uncased' , num_labels =2 ,
                                                   output_attentions=False , 
                                                   output_hidden_states = False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [22]:
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [23]:
optimizer = AdamW(model.parameters() ,lr=2e-5 , eps =1e-8)
epochs = 4
all_steps  = len(train_dataloader) * epochs


schedual = get_linear_schedule_with_warmup(optimizer , num_warmup_steps =0 ,
                                     num_training_steps = all_steps)



In [ ]:

seed_value = 42 
random.seed(seed_value)
np.random.seed(seed_value)
torch.manual_seed(seed_value)
torch.cuda.manual_seed_all(seed_value)



total_loss =[]
for i in range(epochs):
    print(f'===== spochs {i+1}=====')
    tic = time.time()
    loss_val = 0
    
            #######################
            #### train model ######
            #######################
    model.train()
    
    for step , batch in enumerate(train_dataloader):
        inp_ids = batch[0].to(device)
        inp_mask = batch[1].to(device)
        inp_labels = batch[2].to(device)
        
        model.zero_grad()
        outputs = model(inp_ids ,token_type_ids = None ,  
                        attention_mask = inp_mask , labels = inp_labels)
        loss = outputs[0]
        loss_val+=loss 
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        schedual.step()
    toc = time.time()    
    av_loss = loss_val / len(train_dataloader)
    total_loss.append(av_loss)
    
    
    print(f'average of loss : {av_loss} time is ----> {tic-toc}')
    
    
            ###########################
            ###### validation #########
            ###########################
    
    print('Running Validation......')
    num_steps =0
    tic_2 = time.time()
    model.eval()
    accuracy = 0 
    
    for batch in (val_dataloader) :
        inp_ids = batch[0].to(device)
        inp_mask = batch[1].to(device)
        inp_labels = batch[2].to(device)
        
        with torch.no_grad():
            outputs = model(inp_ids , attention_mask=inp_mask , token_type_ids =None)

            prediction = outputs[0]

            out_1 = np.argmax(prediction , axis=1).flatten()
            out_2 = inp_labels.flatten()
            acc = np.sum(out_1==out_2) / len(inp_labels)
            accuracy+=acc
            num_steps+=1

    toc_2 = time.time()
    print(f'accurcy is {accuracy / num_steps} time is ---->{tic_2 -toc_2} ')
    
print('Completed .')
 

===== spochs 1=====
